In [1]:
import numpy as np
import datetime
from IPython.display import display, clear_output

Loading and starting ib_insync

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=4)

<IB connected to 127.0.0.1:7496 clientId=4>

Loading and starting the Redis database.

The following line runs a script that contains the following two lines:
1. import redis
2. red = redis.Redis(password='mypassword')

In [3]:
%run redis_start.py

In [4]:
spy_last60 = np.zeros(shape = (60)) + np.nan
qqq_last60 = np.zeros(shape = (60)) + np.nan

In [5]:
spy_last60 

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])

In [6]:
def make_trade(spy_sign, spy_bid, spy_ask, qqq_bid, qqq_ask):
            
    if spy_sign==1:
       order1 = LimitOrder('BUY',  100, spy_ask+.05)
       order2 = LimitOrder('SELL', 120, qqq_bid-.05)
       print('bought 100 shares of SPY and sold 120 shares of QQQ')
        
    if spy_sign==-1:
       order1 = LimitOrder('SELL', 100, spy_bid-.05)
       order2 = LimitOrder('BUY',  120, qqq_ask+.05)
       print('sold 100 shares of SPY and bought 120 shares of QQQ')
        
    order1.outsideRth = True
    order2.outsideRth = True
    
    contract1 = Stock('SPY', 'SMART', 'USD')
    contract2 = Stock('QQQ', 'SMART', 'USD')

    ib.placeOrder(contract1, order1)
    ib.placeOrder(contract2, order2)

In [9]:
current_spy_position = 0

time_of_last_trade   = datetime.datetime.now()

cutoff = .001

red.set_response_callback('GET',float)

i = 0;
time_of_last_save = datetime.datetime.now()
while i<40000:
   i = i + 1;

   ib.sleep(.1)

   spy_bid = red.get('SPY_bid')
   spy_ask = red.get('SPY_ask')
   qqq_bid = red.get('QQQ_bid')
   qqq_ask = red.get('QQQ_ask')
    
   spy_mid = (spy_bid + spy_ask)/2
   qqq_mid = (qqq_bid + qqq_ask)/2
    
   if (current_spy_position==0):
      #& ((datetime.datetime.now() - time_of_last_trade).total_seconds() > 10):       

      spy_ret = spy_mid / spy_last60[0] - 1
      qqq_ret = qqq_mid / qqq_last60[0] - 1
    
      retdiff = spy_ret - qqq_ret
    
      if retdiff > cutoff:
         make_trade(-1, spy_bid, spy_ask, qqq_bid, qqq_ask)
         current_spy_position = -100
         time_of_last_trade   = datetime.datetime.now()
      if retdiff < -cutoff:
         make_trade(1, spy_bid, spy_ask, qqq_bid, qqq_ask)
         current_spy_position = 100
         time_of_last_trade   = datetime.datetime.now()
            
   else:
    
      if (datetime.datetime.now() - time_of_last_trade).total_seconds() > 60:       
         if current_spy_position==100:  
            make_trade(-1, spy_bid, spy_ask, qqq_bid, qqq_ask)
            current_spy_position = 0
            time_of_last_trade   = datetime.datetime.now()
         if current_spy_position==-100:         
            make_trade(1, spy_bid, spy_ask, qqq_bid, qqq_ask)
            current_spy_position = 0
            time_of_last_trade   = datetime.datetime.now()
        
    
   if (datetime.datetime.now() - time_of_last_save).total_seconds() > 1:       
        
      time_of_last_save = datetime.datetime.now()
    
      spy_last60 = np.delete(spy_last60,0)
      spy_last60 = np.append(spy_last60, spy_mid)

      qqq_last60 = np.delete(qqq_last60,0)
      qqq_last60 = np.append(qqq_last60, qqq_mid)

      clear_output(wait=True)
      display(spy_last60)        
      display(qqq_last60)        
    
    
    

array([361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51, 361.51,
       361.51, 361.51, 361.51, 361.51])

array([295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58, 295.58,
       295.58, 295.58, 295.58, 295.58])

KeyboardInterrupt: 

In [10]:
ib.disconnect()